In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

df_train = pd.read_csv("E:\\leonhardt\\mayi\\data\\train.csv")
df_test = pd.read_csv("E:\\leonhardt\\mayi\\data\\test_a.csv")

In [2]:
df_train = df_train[df_train["label"]!=-1]

In [3]:
count = df_train.describe().loc["count", :]

In [4]:
count = count.sort_values()
count_col = count.index[0:12]
count_col

Index(['f45', 'f38', 'f37', 'f36', 'f41', 'f42', 'f43', 'f44', 'f46', 'f47',
       'f39', 'f40'],
      dtype='object')

In [5]:
df_train.drop(count_col, axis=1, inplace=True)
df_test.drop(count_col, axis=1, inplace=True)

In [6]:
train_set = df_train[df_train["date"]<=20171005]
eva_set = df_train[df_train["date"]>20171005]

In [7]:
train_label = train_set["label"]
eva_label = eva_set["label"]

In [8]:
train_set.drop(["id", "label", "date"], axis=1, inplace=True)
eva_set.drop(["id", "label", "date"], axis=1, inplace=True)

D:\anaconda\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [9]:
train_set.columns

Index(['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       ...
       'f288', 'f289', 'f290', 'f291', 'f292', 'f293', 'f294', 'f295', 'f296',
       'f297'],
      dtype='object', length=285)

In [10]:
for col in train_set.columns:
    train_set[col].fillna(train_set[col].mean(axis=0), inplace=True)

D:\anaconda\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [11]:
for col in eva_set.columns:
    eva_set[col].fillna(eva_set[col].mean(axis=0), inplace=True)

D:\anaconda\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [12]:
df_test.drop(["id", "date"], axis=1, inplace=True)

In [13]:
for col in df_test.columns:
    df_test[col].fillna(df_test[col].mean(axis=0), inplace=True)

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
gbdt = GradientBoostingClassifier(  
    init=None,  
    learning_rate=0.05,  
    loss='deviance',
    max_depth=5,  
    max_features=None,  
    max_leaf_nodes=None,  
    min_samples_leaf=1,  
    min_samples_split=2,  
    min_weight_fraction_leaf=0.0,  
    n_estimators=300,  
    random_state=None,  
    subsample=1.0,  
    verbose=10,  
    warm_start=False)  

In [18]:
gbdt.fit(train_set, train_label)

      Iter       Train Loss   Remaining Time 
         1           0.1025           48.10m
         2           0.0987           49.05m
         3           0.0958           49.26m
         4           0.0932           48.38m
         5           0.0911           47.93m
         6           0.0894           47.46m
         7           0.0876           47.30m
         8           0.0862           47.02m
         9           0.0847           46.80m
        10           0.0834           46.52m
        11           0.0823           46.29m
        12           0.0812           46.35m
        13           0.0802           46.33m
        14           0.0785           46.56m
        15           0.0773           46.56m
        16           0.0759           46.63m
        17           0.0746           46.65m
        18           0.0734           46.60m
        19           0.0724           46.61m
        20           0.0713           46.60m
        21           0.0704           46.54m
        2

       183           0.0430           21.04m
       184           0.0429           20.85m
       185           0.0429           20.65m
       186           0.0429           20.50m
       187           0.0428           20.30m
       188           0.0428           20.12m
       189           0.0427           19.95m
       190           0.0426           19.76m
       191           0.0426           19.55m
       192           0.0425           19.36m
       193           0.0424           19.17m
       194           0.0424           18.98m
       195           0.0423           18.83m
       196           0.0423           18.65m
       197           0.0423           18.44m
       198           0.0423           18.25m
       199           0.0423           18.05m
       200           0.0422           17.85m
       201           0.0422           17.67m
       202           0.0421           17.48m
       203           0.0421           17.31m
       204           0.0421           17.15m
       205

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=300,
              presort='auto', random_state=None, subsample=1.0, verbose=10,
              warm_start=False)

In [26]:
eva_prob = gbdt.predict_proba(eva_set)

In [27]:
from sklearn.metrics import roc_curve
def atec_metric(preds, labels):
    fpr,tpr,threshhold=roc_curve(labels, preds)
    return 'atec_metric', 0.4*tpr[fpr<=0.001][-1] + 0.3*tpr[fpr<=0.005][-1] + 0.3*tpr[fpr<=0.01][-1], True

In [28]:
eva_score = atec_metric(eva_prob[:, 1], eva_label)
eva_score

('atec_metric', 0.292016473942658, True)

In [32]:
eva_prob[:, 1].min()

0.00013609447166257503

In [35]:
np.savetxt(os.getcwd()+"/prediction/gbdt_half_mean_300.txt", eva_prob, delimiter=',')

In [39]:
# df_eval = pd.Series(index=range(10, 380, 30))
# for num in range(10, 380, 30):
gbdt = GradientBoostingClassifier(  
    init=None,  
    learning_rate=0.05,  
    loss='deviance',
    max_depth=5,  
    max_features=None,  
    max_leaf_nodes=None,  
    min_samples_leaf=1,  
    min_samples_split=2,  
    min_weight_fraction_leaf=0.0,  
    n_estimators=160,  
    random_state=None,  
    subsample=1.0,  
    verbose=0,  
    warm_start=False)  
gbdt.fit(train_set, train_label)
eva_prob = gbdt.predict_proba(eva_set)
eva_score = atec_metric(eva_prob[:, 1], eva_label)
print("{0} score is {1}".format(160, eva_score))

160 score is ('atec_metric', 0.3255979724378267, True)


In [40]:
y_preds = gbdt.predict_proba(df_test)

In [41]:
np.savetxt(os.getcwd()+"/prediction/gbdt_half_mean_160.txt", y_preds, delimiter=',')